In [1]:
from __future__ import annotations

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
# Load the important classes
import random as rnd

from env.environment import OnlineFlexibleResourceAllocationEnv
from agents.resource_weighting_agent import ResourceWeightingAgent
from agents.task_pricing_agent import TaskPricingAgent

In [3]:
# Setup environment
env = OnlineFlexibleResourceAllocationEnv.make('../settings/basic_env_setting.json')

In [4]:
# Setup the agents
task_pricing_agents = [TaskPricingAgent('TPA {}'.format(agent_num)) for agent_num in range(10)]
resource_weighting_agents = [ResourceWeightingAgent('RWA {}'.format(agent_num)) for agent_num in range(10)]

In [5]:
# Reset the environment
state = env.reset()
env

Env State (0x19d9815b728)
Auction Task -> None
Servers
	Basic 0 Server - Storage cap: 258, Comp cap: 23, Bandwidth cap: 19, Tasks: []
	Basic 1 Server - Storage cap: 207, Comp cap: 21, Bandwidth cap: 23, Tasks: []
	Basic 2 Server - Storage cap: 275, Comp cap: 24, Bandwidth cap: 20, Tasks: []
	Basic 3 Server - Storage cap: 214, Comp cap: 21, Bandwidth cap: 22, Tasks: []
	Basic 4 Server - Storage cap: 261, Comp cap: 26, Bandwidth cap: 20, Tasks: []

Unallocated tasks
	Basic 6 Task (0x19d97e0b678) - Unassigned, Storage: 67, Comp: 26, Results data: 12, Auction time: 3, Deadline: 9
	Basic 21 Task (0x19d97e0bee8) - Unassigned, Storage: 58, Comp: 35, Results data: 24, Auction time: 7, Deadline: 17
	Basic 19 Task (0x19d97e0bdc8) - Unassigned, Storage: 50, Comp: 41, Results data: 16, Auction time: 10, Deadline: 17
	Basic 20 Task (0x19d97e0be58) - Unassigned, Storage: 77, Comp: 35, Results data: 26, Auction time: 11, Deadline: 19
	Basic 2 Task (0x19d97e0b288) - Unassigned, Storage: 78, Comp: 48, 

In [15]:
state

Env State (0x19d981ea0e8)
Auction Task -> None
Servers
	Basic 0 Server - Storage cap: 258, Comp cap: 23, Bandwidth cap: 19, Tasks: []
	Basic 1 Server - Storage cap: 207, Comp cap: 21, Bandwidth cap: 23, Tasks: [Basic 6]
	Basic 2 Server - Storage cap: 275, Comp cap: 24, Bandwidth cap: 20, Tasks: []
	Basic 3 Server - Storage cap: 214, Comp cap: 21, Bandwidth cap: 22, Tasks: []
	Basic 4 Server - Storage cap: 261, Comp cap: 26, Bandwidth cap: 20, Tasks: []


In [7]:
server_task_pricing_agents = {server: rnd.choice(task_pricing_agents) 
                              for server in state.server_tasks.keys()}
server_resource_allocation_agents = {server: rnd.choice(resource_weighting_agents) 
                                     for server in state.server_tasks.keys()}

In [9]:
auction_trajectory = []
resource_allocation_trajectory = []

In [13]:
actions = {
    server: server_task_pricing_agents[server].price(state.auction_task, server,
                                                     state.server_tasks[server], env.time_step)
    for server in state.server_tasks.keys()
}

print('\n'.join([f'{str(server)} -> {price}' for server, price in actions.items()]))

next_state, rewards, done, info = env.step(actions)

auction_trajectory.append((state, rewards))

print(f'Next State: {str(next_state)}')
print(f'Rewards: {rewards}')
print(f'Done: {done}')

state = next_state

Basic 0 Server - Storage cap: 258, Comp cap: 23, Bandwidth cap: 19 -> 14
Basic 1 Server - Storage cap: 207, Comp cap: 21, Bandwidth cap: 23 -> 3
Basic 2 Server - Storage cap: 275, Comp cap: 24, Bandwidth cap: 20 -> 19
Basic 3 Server - Storage cap: 214, Comp cap: 21, Bandwidth cap: 22 -> 19
Basic 4 Server - Storage cap: 261, Comp cap: 26, Bandwidth cap: 20 -> 26
Next State: Env State (0x19d981ea0e8)
Auction Task -> None
Servers
	Basic 0 Server - Storage cap: 258, Comp cap: 23, Bandwidth cap: 19, Tasks: []
	Basic 1 Server - Storage cap: 207, Comp cap: 21, Bandwidth cap: 23, Tasks: [Basic 6]
	Basic 2 Server - Storage cap: 275, Comp cap: 24, Bandwidth cap: 20, Tasks: []
	Basic 3 Server - Storage cap: 214, Comp cap: 21, Bandwidth cap: 22, Tasks: []
	Basic 4 Server - Storage cap: 261, Comp cap: 26, Bandwidth cap: 20, Tasks: []

Rewards: {Server(name='Basic 1', storage_cap=207, comp_cap=21, bandwidth_cap=23): inf}
Done: False


NameError: name 'next_State' is not defined

In [16]:
actions = {
    server: {
        task: server_resource_allocation_agents[server].weight(server, task, state.server_tasks[server], env.time_step)
        for task in tasks
    }
    for server, tasks in state.server_tasks.items()
}

print('\n'.join([f"{server.name} - [{', '.join([f'{task.name}: {actions[server][task]}' for task in tasks])}]"
                for server, tasks in actions.items()]))

next_state, rewards, done, info = env.step(actions)

# If rewards is a Dict[Server, Dict[Task, float]] is the completed tasks
resource_allocation_trajectory.append((state, next_state, rewards))

print(f'Next State: {str(next_state)}')
print(f'Rewards: {rewards}')
print(f'Done: {done}')

state = next_state

AttributeError: 'ResourceWeightingAgent' object has no attribute 'weight'